In [1]:
import os
import cudf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

#corresponding to model
import tensorflow as tf
from tensorflow.keras.layers import Embedding, GRU, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
 class Metesre():
        
        def __init__(self):
            
            
            item_ids = cudf.read_parquet('/kaggle/input/training-uk-nopadding/categories/unique.prev_items.parquet')
            self.items = item_ids['prev_items'] 
            self.INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/kaggle/input/training-uk-nopadding")
            self.sessions_gdf = pd.read_parquet(os.path.join(self.INPUT_DATA_DIR, "processed_nvt/part_0_filtered_20.parquet"))
            self.test_gdf = pd.read_parquet(os.path.join(self.INPUT_DATA_DIR, "processed_nvt/test_0.parquet"))
            self.preprocessing()
            self.buildModel()
            
            
            
        def preprocessing(self):
            
            
            X1 = self.sessions_gdf['prev_items-list'].tolist()
            X2 = self.sessions_gdf['title-list'].tolist()
            X3 = self.sessions_gdf['brand-list'].tolist()
            X4 = self.sessions_gdf['size-list'].tolist()
            X5 = self.sessions_gdf['model-list'].tolist()
            X6 = self.sessions_gdf['color-list'].tolist()
            
            #handles variable length session sequences
            X1 = np.array(X1, dtype='object')

            #find vocab sizes
            self.vocab_size1 = max(item for sublist in X1 for item in sublist)+1
            self.vocab_size2 = max(item for sublist in X2 for item in sublist)+1
            self.vocab_size3 = max(item for sublist in X3 for item in sublist)+1
            self.vocab_size4 = max(item for sublist in X4 for item in sublist)+1
            self.vocab_size5 = max(item for sublist in X5 for item in sublist)+1
            self.vocab_size6 = max(item for sublist in X6 for item in sublist)+1
            
            print("Vocab Sizes: \n",self.vocab_size1, self.vocab_size2, self.vocab_size3, self.vocab_size4, self.vocab_size5, self.vocab_size6)
            
            #extract next item from the X1: prev_items_list, also remove last items attributes
            X1_p = []
            X2_p = []
            X3_p = []
            X4_p = []
            X5_p = []
            X6_p = []

            y_p = []

            for i in range(len(X1)):
                X1_p.append(X1[i][:-1])
                X2_p.append(X2[i][:-1])
                X3_p.append(X3[i][:-1])
                X4_p.append(X4[i][:-1])
                X5_p.append(X5[i][:-1])
                X6_p.append(X6[i][:-1])
                y_p.append(X1[i][-1])
                
        
            X1 = X1_p
            X2 = X2_p
            X3 = X3_p
            X4 = X4_p
            X5 = X5_p
            X6 = X6_p
            y= y_p
            y = np.array(y)
            
            #padding: pre for X1 and post for all others
            X1 = pad_sequences(X1, maxlen=200, padding='pre')
            X2 = pad_sequences(X2, maxlen=200, padding='pre')
            X3 = pad_sequences(X3, maxlen=200, padding='pre')
            X4 = pad_sequences(X4, maxlen=200, padding='pre')
            X5 = pad_sequences(X5, maxlen=200, padding='pre')
            X6 = pad_sequences(X6, maxlen=200, padding='pre')

            self.X1_train, self.X1_test, self.X2_train, self.X2_test, self.X3_train, self.X3_test, self.X4_train, \
            self.X4_test, self.X5_train, self.X5_test, self.X6_train, self.X6_test, self.y_train, self.y_test = train_test_split(X1, X2, X3, X4, \
                                                                                              X5, X6, y, test_size=0.05,random_state=42)
        
        def buildModel(self):
            
            embedding_dim = 128
            hidden_units = 128
            seq_length = 200

            # Define the input layers
            input_layer1 = tf.keras.Input(shape=(seq_length,))
            input_layer2 = tf.keras.Input(shape=(seq_length,))
            input_layer3 = tf.keras.Input(shape=(seq_length,))
            input_layer4 = tf.keras.Input(shape=(seq_length,))
            input_layer5 = tf.keras.Input(shape=(seq_length,))
            input_layer6 = tf.keras.Input(shape=(seq_length,))

            # Define the embedding layers
            embedding_layer1 = Embedding(self.vocab_size1, embedding_dim)
            embedding_layer2 = Embedding(self.vocab_size2, embedding_dim)
            embedding_layer3 = Embedding(self.vocab_size3, embedding_dim)
            embedding_layer4 = Embedding(self.vocab_size4, embedding_dim)
            embedding_layer5 = Embedding(self.vocab_size5, embedding_dim)
            embedding_layer6 = Embedding(self.vocab_size6, embedding_dim)
            

            # Define the GRU layer
            gru_layer = GRU(hidden_units, return_sequences=False)

            # Define the output layer
            output_layer = Dense(self.vocab_size1, activation='softmax')


            # Connect the layers
            embedded_input1 = embedding_layer1(input_layer1)
            embedded_input2 = embedding_layer2(input_layer2)
            embedded_input3 = embedding_layer3(input_layer3)
            embedded_input4 = embedding_layer4(input_layer4)
            embedded_input5 = embedding_layer5(input_layer5)
            embedded_input6 = embedding_layer6(input_layer6)


            # Concatenate the outputs of the two GRU layers
            concatenated_output = Concatenate()([embedded_input1, embedded_input2, embedded_input3, embedded_input4, embedded_input5, embedded_input6])

            gru_output = gru_layer(concatenated_output)


            output = output_layer(gru_output)


            # Create the model
            self.model = Model(inputs=[input_layer1, input_layer2, input_layer3, input_layer4, input_layer5, input_layer6], outputs=output)

            # Compile the model
            self.model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[tf.keras.metrics.CosineSimilarity(axis=1)])

            # Print the model summary
            self.model.summary()
            
            
        def _mean_reciprocal_rank(self, recommendations, ground_truth):
            """
            Calculate the Mean Reciprocal Rank (MRR) of a recommendation system.

            :param recommendations: A list of lists containing the recommended items for each query.
            :param ground_truth: A list containing the ground truth (relevant) items for each query.
            :return: The Mean Reciprocal Rank (MRR) value as a float.
            """
            assert len(recommendations) == len(ground_truth), "Recommendations and ground truth lists must have the same length."

            reciprocal_ranks = []

            for rec, gt in zip(recommendations, ground_truth):
                for rank, item in enumerate(rec, start=1):
                    if item == gt:
                        reciprocal_ranks.append(1 / rank)
                        break
                else:
                    reciprocal_ranks.append(0)

            mrr = sum(reciprocal_ranks) / len(reciprocal_ranks)
            return mrr


        def train(self, epoch= 10, batch_size = 32):
            
            checkpoint_callback = ModelCheckpoint(
                '/kaggle/working/model_checkpoint.h5',
                monitor='val_loss',  
                save_best_only=True,  
                save_weights_only=False, 
                verbose=1 
            )
            
            self.history = self.model.fit([ self.X1_train, self.X2_train, self.X3_train, self.X4_train, self.X5_train, self.X6_train], self.y_train, epochs = epoch, batch_size=batch_size, verbose = True, validation_split=0.1, callbacks=[checkpoint_callback])
            
        
        
        
        def _decoder(self, recommendation):
            
            '''decode sequeces to ASIN ids'''
            
            decoded = []
            for next_item in recommendation:

                decoded.append([self.items.iloc[e] for e in next_item])

            decoded = np.array(decoded)
            return decoded
            
            
            
        def _predictor(self, X_test):
            
            '''generate y_pred (which is top 100 product indices) from the model for X_test. '''
            
            batch_size = 64
            num_batches = int(len(X_test[0]) / batch_size)

            y_pred = []
            for batch_idx in range(num_batches+1):
                
                if batch_idx < num_batches:
                    start_idx = batch_idx * batch_size
                    end_idx = (batch_idx + 1) * batch_size
                    
                    inputs = []
                
                    for i in range(len(X_test)):
                        inputs.append(X_test[i][start_idx:end_idx])
                        
                    predictions = self.model.predict(inputs)
                    recom_size = 100
            
            
                    top_preds = np.argpartition(predictions, -recom_size, axis=1)[:, -recom_size:]
                    sorted_indices = np.argsort(predictions[np.arange(len(predictions))[:, None], top_preds], axis=1)[:, ::-1]
                    recom = top_preds[np.arange(len(predictions))[:, None], sorted_indices]

                    y_pred.append(recom)

                        
                else:
                    
                    inputs = []
                
                    for i in range(len(X_test)):
                        
                        inputs.append(X_test[i][end_idx:])
                    
                    predictions = self.model.predict(inputs)
            
                    top_preds = np.argpartition(predictions, -recom_size, axis=1)[:, -recom_size:]
                    sorted_indices = np.argsort(predictions[np.arange(len(predictions))[:, None], top_preds], axis=1)[:, ::-1]
                    recom = top_preds[np.arange(len(predictions))[:, None], sorted_indices]

                    y_pred.append(recom)
                
            y_pred = [inner_list for outer_list in y_pred for inner_list in outer_list]
                    
            return y_pred
            
            
        def test_1_testontest(self):
            
            
            '''evaluate model's performance on the test set defined in the initialization '''
            #update it for all the test sessions instead of only 200
            
            recommendation = self._predictor([self.X1_test, self.X2_test, self.X3_test, self.X4_test, self.X5_test, self.X6_test])
            gnd = self.y_test.tolist()
            self.test1_MRR = self._mean_reciprocal_rank(recommendation, gnd)
            print(f'MRR for test1: {self.test1_MRR}')

    
        
            
        def test_2_testwithendone(self, n=100):
            
            ''' evaluate model's performance on the given test set. Since test set has no ground truth
            we will split the last item in the session and consider it as the next item and evaulate model
            performance'''
            
            
            X1 = self.test_gdf['prev_items-list'].tolist()
            X2 = self.test_gdf['title-list'].tolist()
            X3 = self.test_gdf['brand-list'].tolist()
            X4 = self.test_gdf['size-list'].tolist()
            X5 = self.test_gdf['model-list'].tolist()
            X6 = self.test_gdf['color-list'].tolist()
            
            #handles variable length session sequences
            X1 = np.array(X1, dtype='object')


            #extract next item from the X1: prev_items_list, also remove last items attributes
            X1_p = []
            X2_p = []
            X3_p = []
            X4_p = []
            X5_p = []
            X6_p = []

            y_p = []

            for i in range(len(X1)):
                X1_p.append(X1[i][:-1])
                X2_p.append(X2[i][:-1])
                X3_p.append(X3[i][:-1])
                X4_p.append(X4[i][:-1])
                X5_p.append(X5[i][:-1])
                X6_p.append(X6[i][:-1])
                y_p.append(X1[i][-1])
                
        
            X1 = X1_p
            X2 = X2_p
            X3 = X3_p
            X4 = X4_p
            X5 = X5_p
            X6 = X6_p
            y= y_p
            y = np.array(y)
            
            #padding: pre for X1 and post for all others
            X1 = pad_sequences(X1, maxlen=200, padding='pre')
            X2 = pad_sequences(X2, maxlen=200, padding='pre')
            X3 = pad_sequences(X3, maxlen=200, padding='pre')
            X4 = pad_sequences(X4, maxlen=200, padding='pre')
            X5 = pad_sequences(X5, maxlen=200, padding='pre')
            X6 = pad_sequences(X6, maxlen=200, padding='pre')
            
           
            rec = self._predictor([X1[:n], X2[:n], X3[:n], X4[:n], X5[:n], X6[:n]])
            gnd = y[:n].tolist()
            self.test2_MRR =  self._mean_reciprocal_rank(rec, gnd)
            print(f'MRR for test1: {self.test2_MRR}')

            
        
    
        def test_3_generatefinalresult(self, n=100):
            
            
            ''' generates predictions of test set. Decodes the index and return the recommendations with ASIN ids'''
            
            X1 = self.test_gdf['prev_items-list'].tolist()
            X2 = self.test_gdf['title-list'].tolist()
            X3 = self.test_gdf['brand-list'].tolist()
            X4 = self.test_gdf['size-list'].tolist()
            X5 = self.test_gdf['model-list'].tolist()
            X6 = self.test_gdf['color-list'].tolist()
 

            
            X1 = pad_sequences(X1, maxlen=200, padding='pre')
            X2 = pad_sequences(X2, maxlen=200, padding='pre')
            X3 = pad_sequences(X3, maxlen=200, padding='pre')
            X4 = pad_sequences(X4, maxlen=200, padding='pre')
            X5 = pad_sequences(X5, maxlen=200, padding='pre')
            X6 = pad_sequences(X6, maxlen=200, padding='pre')
            
          
            rec = self._predictor([X1[:n], X2[:n], X3[:n], X4[:n], X5[:n], X6[:n]])

            y_pred = self._decoder(rec)
            y_pred = y_pred.tolist()
            df = pd.DataFrame()
            df['next_item_prediction'] = y_pred
            
            return df
            

In [4]:
ob = Metesre()
# plot_model(ob.model)

Vocab Sizes: 
 499121 581870 78971 88548 201795 89095
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 200)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 200)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 200)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 200)]        0           []                               
                                        

In [5]:
ob.train(epoch = 2)

Epoch 1/2
1517/1517 [==============================] - ETA: 0s - loss: 12.0407 - cosine_similarity: 367.2119
Epoch 1: val_loss improved from inf to 11.12500, saving model to /kaggle/working/model_checkpoint.h5
1517/1517 [==============================] - 286s 184ms/step - loss: 12.0407 - cosine_similarity: 367.2119 - val_loss: 11.1250 - val_cosine_similarity: 264.1793
Epoch 2/2
1517/1517 [==============================] - ETA: 0s - loss: 8.2602 - cosine_similarity: 57.6658
Epoch 2: val_loss did not improve from 11.12500
1517/1517 [==============================] - 104s 68ms/step - loss: 8.2602 - cosine_similarity: 57.6658 - val_loss: 11.1590 - val_cosine_similarity: 60.0115


In [9]:
y_pre = ob.test_2_testwithendone(n = 1000)

2/2 [==============================] - 0s 8ms/step
MRR for test1: 0.4512223893554617


In [7]:
ob.test_1_testontest()

1/1 [==============================] - 0s 28ms/step
MRR for test1: 0.1703026447944764


In [8]:
y_pre = ob.test_3_generatefinalresult()
y_pre

2/2 [==============================] - 0s 12ms/step


,next_item_prediction
0,"[B01HTOD43Y, B08F67XG18, B07BYCHMVY, B08HLJP1C..."
1,"[B09GLLZFK2, B094R6BDN6, B09CLG5WKR, B0BFFSRNG..."
2,"[B07KCKWGSG, B0865RL4PH, B095J18TP6, B00HZV9YE..."
3,"[B01GN0M6NE, B09BCG9Z17, B08YK494ZZ, B09W4CYBT..."
4,"[B007CVCVUM, B09XBF84Y7, B014WLL3UI, B07H7915Z..."
...,...
95,"[B08TCH3HSR, B01KZC1EAW, B099N5CWRF, B09HGYSFG..."
96,"[B07P57Z363, B08FHSBDNC, B08R5BKKM6, B004ISW5Y..."
97,"[B08HN5YZS4, B07P57Z363, B0B881WK35, B07L4S22N..."
98,"[B01BRDU22O, B07VM92YRB, B075JJ5615, B0B7BDN73..."
